In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool
from rtree import index
import datetime

In [2]:
mc_data = pd.read_json('DataSets/SciFi_200000_Clean_TrackStudy.json')

In [3]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [4]:
mc_hits = mc_data.values
print('Shape MC_Hits: %s' % str(mc_hits.shape))

Shape MC_Hits: (200000, 14)


In [5]:
all_hits = np.zeros((3000000, 11))
i = 0
empty_particles = []
for hits in mc_hits:
    if (len(hits[4]) > 1):
        for j in range(0, len(hits[4])):
            hit = np.zeros(11)
            for k in range(0,9):
                hit[k + 1] = hits[k + 4][j]
            hit[10] = hits[13]
            hit[0] = hits[0]
            all_hits[i] = hit
            i += 1
            
# Removing empty values, as some hits in the mc_hits dataset do not contain readings (PrHit...)
all_hits = all_hits[all_hits[:,0] != 0]
all_hits.shape

(2025994, 11)

In [6]:
# Parameters for the 2-hit combination
L0_AlphaCorr = [120.64 , 510.64 , 730.64]
L0_tolHp = [280.0 , 540.0 , 1080.0]

# Parameters for the 3-hit combination
x0Corr = [0.002152 , 0.001534 , 0.001834]
X0SlopeChange = [400. , 500. , 500.]
x0Cut = [1500.0 , 4000. , 6000.]
TolAtX0Cut = [4.5 , 8.0 , 14.0]
ToleranceX0Up = [0.75, 0.75 , 0.75]
X0SlopeChangeDown = [2000.0 , 2000.0 , 2000.0]
TolAtX0CutOpp = [0.75 , 2.0 , 7.0]
ToleranceX0Down = [0.75 , 0.75 , 0.75]
maxParabolaSeedHits = 12

# Extra constants
z_last = np.unique(all_hits[all_hits[:,5] == 11][:,3])[0]

def sort_xt2(x, y):
    return np.abs(x[2] - y[2])

In [82]:
# Rtree setup
idx = index.Index()
for hit in all_hits:
    idx.insert(int(hit[1]), (float(hit[2]), float(hit[3]), float(hit[2]), float(hit[3])), hit)

212


In [132]:
event_hits = all_hits[(all_hits[:,10] == 15837)]

list(idx.nearest((event_hits[0][2], 7825.99951172, event_hits[0][2] + 200, 7825.99951172), 2))

[316449, 316222, 315986, 316219]

In [131]:
np.unique(all_hits[all_hits[:,5] == 0][:,3])[0]

7825.9995117188

In [103]:
def work(identifier):
    i = 0
    start_time = datetime.datetime.now()
    events = all_events[toSplit*identifier:toSplit*(identifier+1)]
    list_founded = np.zeros(len(events))
    # For each event
    for event in events:
        # Get all hits from a given event
        event_hits = all_hits[(all_hits[:,10] == event)]
        found = []
        found_tmp = []
        for case in range(0,3):
            
            S1 = (TolAtX0Cut[case] - ToleranceX0Up[case]) / (x0Cut[case] - X0SlopeChange[case])  
            S2 = (TolAtX0CutOpp[case] - ToleranceX0Down[case]) / (x0Cut[case] - X0SlopeChangeDown[case])
            if case > 1:
                first = 3
            else:
                first = 0
                
            if case == 1:
                third = 8
            else:
                third = 11
            
            # Upper part and then lower
            for k in range(0,2):
                fist_layer = event_hits[(event_hits[:,5] == first) & (event_hits[:,9] == k)]
                last_layer = event_hits[(event_hits[:,5] == third) & (event_hits[:,9] == k)]
                middle_layer = event_hits[(event_hits[:,5] == 4) & (event_hits[:,9] == k)]
                for hit in fist_layer:
                    collected = []
                    m = hit[2] / hit[3]
                    x_pred = hit[2] + m * ((z_last - hit[3]) +  L0_AlphaCorr[case])
                    for temp in last_layer:
                        if (np.abs(temp[2] - x_pred) < L0_tolHp[case]):
                            collected.append(temp)
                    for temp in collected:
                        m_tmp = (temp[2] - hit[2]) / (temp[3] - hit[3])
                        x0 = hit[2] - hit[3] * m_tmp
                        x_pred_tmp = (hit[2] + m_tmp * (temp[3] - hit[3])) + (x0 * x0Corr[case])
                        Bl = 0
                        Bh = 0
                        if (x0 > 0):
                            if (x0 > X0SlopeChange[case]):
                                bl = -S1 * (x0 - X0SlopeChange[case])
                                bh = S2 * (x0 - X0SlopeChangeDown[case])
                            else:
                                bl = -ToleranceX0Up[case]
                                bh = ToleranceX0Down[case]
                        else:
                            if (x0 < -X0SlopeChange[case]):
                                bl = -S1 * (x0 + X0SlopeChange[case])
                                bh = S2 * (x0 +  X0SlopeChangeDown[case])
                            else:
                                bl = -ToleranceX0Down[case]
                                bh = ToleranceX0Up[case]
                        
                        for temp_second in middle_layer:
                            if (bl < (temp_second[2] - x_pred_tmp) and (temp_second[2] - x_pred_tmp) < bh):
                                toStore = {}
                                toStore['fist_hit'] = hit
                                toStore['second_hit'] = temp_second
                                toStore['third_hit'] = temp
                                new_pred = 0
                                m = (temp_second[2] - hit[2]) / (temp_second[3] - hit[3]) 
                                new_x_pred = hit[2] + m * ((z_last - hit[3]) + L0_AlphaCorr[case])
                                max = np.max(last_layer[:,2])
                                for temp in last_layer:
                                    if (np.abs(temp[2] - new_x_pred) < max):
                                        toStore['new_pred'] = temp[1]
                                        max = np.abs(temp[2] - new_x_pred)
                                found_tmp.append(toStore)
                        if len(found_tmp) > 0:
                            found_tmp = sorted(found_tmp, key = lambda p: np.abs(p['second_hit'][2] - x_pred_tmp))[:12]
            found.append(found_tmp)
        events_tmp = mc_hits[mc_hits[:,13] == event]
        founded = 0
    end = datetime.datetime.now()
    print("Thread %d took: %d ms" % (identifier, (end - start_time).microseconds / 1000))
    return found

In [104]:
all_events = np.unique(mc_hits[:,13])[:7]
toSplit = int(np.round(len(all_events) / 7))

pool = Pool(processes=7)
#map doWork to availble Pool processes
results = pool.map(work, ( 0, 1, 2, 3, 4, 5, 6))
pool.close()
pool.join()      

Thread 6 took: 429 ms
Thread 5 took: 440 ms
Thread 0 took: 97 ms
Thread 1 took: 172 ms
Thread 3 took: 616 ms
Thread 2 took: 497 ms
Thread 4 took: 36 ms


In [111]:
results

[[[{'fist_hit': array([   211.        , 317360.        ,   1081.11010742,   7825.99951172,
                1.        ,      0.        ,     34.60207367,   2425.36230469,
                1.37809765,      1.        ,  15837.        ]),
    'new_pred': 1037227.0,
    'second_hit': array([    11.        , 579284.        ,   1137.29003906,   8507.99902344,
                1.        ,      4.        ,     34.60207367,   2425.36230469,
                1.37809765,      1.        ,  15837.        ]),
    'third_hit': array([     11.        , 1036584.        ,    1136.79003906,
              9403.00097656,       1.        ,      11.        ,
                34.60207367,    2424.60620117,       0.62188935,
                 1.        ,   15837.        ])},
   {'fist_hit': array([   211.        , 280131.        ,    412.63500977,   7825.99951172,
                1.        ,      0.        ,     34.60207367,     -0.62188935,
            -2424.60620117,      0.        ,  15837.        ]),
    'new_pr

In [108]:
print(len(events))

NameError: name 'events' is not defined